# Data Preprocessing Template

## Importing the libraries

In [ ]:
import os
import numpy as np
import pandas as pd

import torch
from PIL import Image
import faiss
from transformers import CLIPProcessor, CLIPModel

import requests
from io import BytesIO

import os
import numpy as np
import faiss
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import gradio as gr

In [ ]:

# Ścieżka do folderu ze zdjęciami
folder_path = r"C:\Users\user\Desktop\CLIP"

# Ładujemy model CLIP (openai/clip-vit-base-patch32)
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Funkcja do ekstrakcji wektorów obrazów
def get_image_embedding(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        embeddings = model.get_image_features(**inputs)
    return embeddings[0].cpu().numpy()


Fetching 1 files: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [ ]:
#Tworzenie csv na podstawie zdjęć

data = {
    "filename": [
        "camera.jpeg",
        "cat.jpeg",
        "coffee.jpeg",
        "girl.jpeg",
        "house_wife.jpeg",
        "motorcycle_right.jpeg",
        "page.png",
        "teacher.jpeg"
    ],
    "label": [
        "camera",
        "cat",
        "coffee",
        "girl",
        "house_wife",
        "motorcycle_right",
        "page",
        "teacher"
    ]
}

df = pd.DataFrame(data)

# zapisujemy do CSV w tym samym katalogu co skrypt
df.to_csv("Data.csv", index=False)

print("Plik Data.csv został utworzony!")


IndentationError: unexpected indent (2048132758.py, line 1)

## FAISS

In [10]:
dataset = pd.read_csv('Data.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values


# Lista plików (tylko obrazy)
valid_ext = [".jpg", ".jpeg", ".png"]
image_files = [
    f for f in os.listdir(folder_path)
    if os.path.splitext(f)[1].lower() in valid_ext
]

print("Znalezione obrazy:", image_files)

# Embeddingi
embeddings = []
labels = []

for img_file in image_files:
    path = os.path.join(folder_path, img_file)
    emb = get_image_embedding(path)
    embeddings.append(emb)
    labels.append(img_file.split(".")[0])


# Embeddingi
embeddings = []
labels = []

for img_file in image_files:
    path = os.path.join(folder_path, img_file)
    emb = get_image_embedding(path)
    embeddings.append(emb)
    labels.append(img_file.split(".")[0])  # np. "camera", "cat"

embeddings = np.array(embeddings).astype("float32")

# Tworzymy index FAISS
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance
index.add(embeddings)

print(f"Załadowano {len(embeddings)} obrazów do FAISS.")


Znalezione obrazy: ['camera.jpeg', 'cat.jpeg', 'coffee.jpeg', 'girl.jpeg', 'house_wife.jpeg', 'motorcycle_right.jpeg', 'page.png', 'teacher.jpeg']
Załadowano 8 obrazów do FAISS.


## First Test

In [ ]:


url = "https://images.unsplash.com/photo-1592194996308-7b43878e84a6"  # przykładowy link do kota
response = requests.get(url)
img = Image.open(BytesIO(response.content)).convert("RGB")

# zapis do folderu CLIP
img.save(r"C:\Users\user\Desktop\CLIP\cat_highres.jpeg")
img.show()


In [ ]:
new_emb = get_image_embedding(r"C:\Users\user\Desktop\CLIP\cat_highres.jpeg").astype("float32")
index.add(new_emb.reshape(1, -1))
labels.append("cat_highres")


In [22]:
# Ścieżka do obrazu, którego używasz jako query
query_image = r"C:\Users\user\Desktop\CLIP\cat_highres.jpeg"

query_emb = get_image_embedding(query_image).astype("float32")
D, I = index.search(query_emb.reshape(1, -1), k=10)  # pobieramy więcej wyników

print("Najbliższe dopasowania (bez query samego siebie):")
for idx, dist in zip(I[0], D[0]):
    if labels[idx] != "cat_highres":  # ignorujemy obraz użyty jako query
        print(f"{labels[idx]} (distance={dist:.4f})")
        break  # jeśli chcemy tylko 1 najlepszy wynik


Najbliższe dopasowania (bez query samego siebie):
cat (distance=66.8705)


In [16]:
folder_path = r"C:\Users\user\Desktop\CLIP"  # folder z Twoimi obrazami
valid_ext = [".jpg", ".jpeg", ".png"]

In [17]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

Fetching 1 files: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [18]:

def get_image_embedding_from_pil(image_pil):
    inputs = processor(images=image_pil, return_tensors="pt")
    with torch.no_grad():
        emb = model.get_image_features(**inputs)
    return emb[0].cpu().numpy()

In [19]:
# 3️⃣ Wczytaj obrazy i stwórz embeddingi
# -----------------------------
image_files = [f for f in os.listdir(folder_path) if os.path.splitext(f)[1].lower() in valid_ext]
embeddings = []
labels = []

for img_file in image_files:
    path = os.path.join(folder_path, img_file)
    img = Image.open(path).convert("RGB")
    emb = get_image_embedding_from_pil(img)
    embeddings.append(emb)
    labels.append(img_file.split(".")[0])

embeddings = np.array(embeddings).astype("float32")


In [20]:
# 4️⃣ Tworzymy indeks FAISS
# -----------------------------
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

print(f"Załadowano {len(embeddings)} obrazów do FAISS.")

Załadowano 8 obrazów do FAISS.


In [21]:
def recognize_image(img: Image.Image):
    # embedding zapytania
    query_emb = get_image_embedding_from_pil(img).astype("float32")
    
    # top-3 podobne obrazy
    D, I = index.search(query_emb.reshape(1, -1), k=3)
    
    # przygotowanie wyników do Gradio (miniaturki + etykieta)
    results = []
    for idx in I[0]:
        result_img = Image.open(os.path.join(folder_path, image_files[idx]))
        results.append((result_img, labels[idx]))
    return results

In [ ]:
# 6️⃣ Interfejs Gradio
# -----------------------------
iface = gr.Interface(
    fn=recognize_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Gallery(label="Najbliższe obrazy"),
    title="CLIP + FAISS Prosta wyszukiwarka"
    description="Prześlij obraz, a model znajdzie podobne obrazy w bazie."
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
